In [1]:
import pandas as pd
import random

# -----------------------------
# LOAD DATA
# -----------------------------
venues_df = pd.read_csv("C:/Users/HP/Documents/AAMUSTED/2025/timetable/input_data/venue_cap.csv") # venue, min_cap, max_cap
courses_df = pd.read_csv("C:/Users/HP/Documents/AAMUSTED/2025/timetable/input_data/courses.csv")
invigilators_df = pd.read_csv("C:/Users/HP/Documents/AAMUSTED/2025/timetable/input_data/invigilators.csv")

# Normalize faculty codes
courses_df["faculty"] = courses_df["faculty"].astype(str).str.upper().str.strip()
invigilators_df["faculty"] = invigilators_df["faculty"].astype(str).str.upper().str.strip()

# Group invigilator initials by faculty
invigilators = (
    invigilators_df.groupby("faculty")["initials"]
    .apply(list)
    .to_dict()
)

# -----------------------------
# DEFINE SCHEDULE PARAMETERS
# -----------------------------
days = [f"Day {i+1}" for i in range(10)]  # replace with calendar dates if needed
sessions = ["7-9", "10-12", "1-3", "4-6"]

# -----------------------------
# VENUE ALLOCATION FUNCTION
# -----------------------------
def find_venue(students):
    # Try to find a venue where capacity fits
    exact_fit = venues_df[
        (venues_df["min_cap"] <= students) & (venues_df["max_cap"] >= students)
    ]
    if not exact_fit.empty:
        return random.choice(exact_fit["venue"].tolist())
    
    # If none fits exactly, try a bigger venue
    bigger = venues_df[venues_df["max_cap"] >= students]
    if not bigger.empty:
        return random.choice(bigger["venue"].tolist())
    
    # Otherwise, split across multiple small venues
    sorted_venues = venues_df.sort_values("max_cap")
    assigned = []
    remaining = students
    for _, v in sorted_venues.iterrows():
        assigned.append(v["venue"])
        remaining -= v["max_cap"]
        if remaining <= 0:
            break
    return " + ".join(assigned) if assigned else "No Venue"


# -----------------------------
# TIMETABLE GENERATION
# -----------------------------
course_list = courses_df.to_dict("records")

schedule_rows = []

day_index = 0
session_index = 0

for row in course_list:
    class_name = row["class"]
    course = row["course_code"]
    title = row["course_title"]
    faculty = row["faculty"].upper().strip()   # from courses.csv
    students = int(row["stud_numb"]) if not pd.isna(row["stud_numb"]) else 0

    venue = find_venue(students)

    # pick an invigilator from the right faculty
    available_invs = invigilators.get(faculty, []) or ["TBD"]
    invigilator = random.choice(available_invs)

    # lookup department of the invigilator
    inv_row = invigilators_df[invigilators_df["initials"] == invigilator]
    dept = inv_row["department"].values[0] if not inv_row.empty else "TBD"

    # Create one row for the timetable
    schedule_rows.append({
        "DAY & DATE": days[day_index],
        "TIME": sessions[session_index],
        "CLASS": class_name,
        "COURSE CODE": course,
        "COURSE TITLE": title,
        "NO. OF STDS": students,
        "VENUE": venue,
        "INVIG.": invigilator,
        "FACULTY": faculty,
        "DEPARTMENT": dept
    })

    # Advance slot
    session_index += 1
    if session_index >= len(sessions):
        session_index = 0
        day_index += 1

# -----------------------------
# CREATE TIMETABLE TABLE
# -----------------------------
timetable_df = pd.DataFrame(schedule_rows)

# Save as Excel
timetable_df.to_excel("exam_timetable_split.xlsx", index=False)
print("✅ Exam timetable with venue splitting saved as exam_timetable_split.xlsx")

# Preview
print(timetable_df.head(10))


✅ Exam timetable with venue splitting saved as exam_timetable_split.xlsx
  DAY & DATE   TIME   CLASS COURSE CODE  \
0      Day 1    7-9  100BIO      BSE122   
1      Day 1  10-12   100SC      ISC122   
2      Day 1    1-3  100CHE      CHE122   
3      Day 1    4-6  100NUD      NUD126   
4      Day 2    7-9  300CHE      CHE363   
5      Day 2  10-12  200BIO      BSE244   
6      Day 2    1-3  200SCI      ISC241   
7      Day 2    4-6  300MAT    MATD 362   
8      Day 3    7-9  300PHY      PHY361   
9      Day 3  10-12  100PHY     MATP121   

                                        COURSE TITLE  NO. OF STDS  \
0                               GENERAL MICROBIOLOGY          187   
1                                            ECOLOGY          149   
2  EQUILIBRIUM, REDOX, ACIDS, BASES, SALT AND THE...           63   
3                        HEALTH SOCIETY AND BEHAVIOR           35   
4  SEPARATION TECHNIQUES AND SPECTROSCOPIC ANALYS...           44   
5          BIODIVERSITY CONSERVATION AN